# 特徴量相関分析 v2

**目的**: 数値特徴量間の相関とターゲットとの相関を分析

**このノートブックで行うこと**:
1. ターゲット変数との相関分析
2. 多重共線性の検出
3. 特徴量選択の示唆を得る

## セットアップ

In [ ]:
import sys
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# プロジェクトルートをパスに追加
project_root = Path().resolve().parent.parent
sys.path.insert(0, str(project_root))
sys.path.insert(0, str(project_root / "04_src"))
os.chdir(project_root)

import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns

# フォント設定
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
try:
    import japanize_matplotlib
    japanize_matplotlib.japanize()
except ImportError:
    plt.rcParams['font.sans-serif'] = ['Hiragino Sans', 'Yu Gothic', 'Meiryo', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

# テスト済み関数のimport
from eda.correlation import calculate_correlations, find_high_correlation_pairs

print("✓ セットアップ完了")

## 1. データ読み込み

In [ ]:
# データ読み込み（DataLoaderパターン）
from src.data.loader import DataLoader
from src.utils.config import load_config

data_config = load_config("data")
loader = DataLoader(data_config)
train = loader.load_train()

print(f"訓練データ: {train.shape}")
train.head()

## 2. 数値カラムの抽出

In [ ]:
# 数値カラムを抽出（Float64、Int64）
numeric_cols = [col for col in train.columns if train[col].dtype in [pl.Float64, pl.Int64]]

# ターゲット変数を除外
target_col = "money_room"
feature_cols = [col for col in numeric_cols if col != target_col]

print(f"数値カラム数: {len(numeric_cols)}")
print(f"特徴量カラム数: {len(feature_cols)}")
print(f"\n先頭10件: {feature_cols[:10]}")

## 3. ターゲット変数との相関分析

In [ ]:
# ターゲットとの相関を計算（テスト済み関数）
target_corr = calculate_correlations(train, target_col, feature_cols)

print("[ターゲット変数との相関 - 上位20件]")
print(target_corr.head(20))

In [ ]:
# 相関係数の可視化（上位30件）
top_30 = target_corr.head(30)

plt.figure(figsize=(10, 12))
plt.barh(range(len(top_30)), top_30['correlation'][::-1])
plt.yticks(range(len(top_30)), top_30['feature'][::-1])
plt.xlabel('相関係数')
plt.title(f'{target_col}との相関（上位30件）')
plt.axvline(x=0, color='k', linestyle='--', linewidth=0.5)
plt.tight_layout()
plt.show()

## 4. 多重共線性の検出

In [ ]:
# 高相関ペアの検出（閾値0.8）
high_corr_pairs = find_high_correlation_pairs(train, feature_cols, threshold=0.8)

print(f"[多重共線性の可能性があるペア: {len(high_corr_pairs)}件]")
if len(high_corr_pairs) > 0:
    for col1, col2, corr in high_corr_pairs[:20]:
        print(f"{col1:40s} - {col2:40s}: {corr:6.3f}")
else:
    print("多重共線性の可能性がある特徴量ペアは見つかりませんでした。")

## 5. 相関ヒートマップ（上位特徴量）

In [ ]:
# ターゲットとの相関が高い上位15特徴量でヒートマップ
top_15_features = target_corr.head(15)['feature'].to_list()
selected_features = [target_col] + top_15_features

# Polarsで相関行列を計算
selected_df = train.select(selected_features).drop_nulls()

# NumPy配列に変換して相関行列を計算
import numpy as np
corr_matrix = np.corrcoef(selected_df.to_numpy(), rowvar=False)

# ヒートマップ作成
plt.figure(figsize=(14, 12))
sns.heatmap(
    corr_matrix,
    annot=True,
    fmt='.2f',
    cmap='coolwarm',
    center=0,
    vmin=-1,
    vmax=1,
    xticklabels=selected_features,
    yticklabels=selected_features,
    square=True,
    cbar_kws={'shrink': 0.8}
)
plt.title('相関ヒートマップ（ターゲット + 上位15特徴量）', pad=20)
plt.tight_layout()
plt.show()

## 6. まとめ

### ターゲットとの相関
- **高相関特徴量（|r| > 0.5）**: [メモ]
- **中程度相関（0.3 < |r| < 0.5）**: [メモ]
- **低相関（|r| < 0.3）**: [メモ]

### 多重共線性
- **高相関ペア数**: X件
- **対応方針**: 
  - [ ] どちらか一方を削除
  - [ ] PCA等で次元削減
  - [ ] そのまま使用（tree-basedモデルは比較的頑健）

### 次のステップ
- [ ] カテゴリ変数の分析（04_categorical_analysis.ipynb）
- [ ] 地理空間分析（05_geospatial_analysis.ipynb）
- [ ] 特徴量エンジニアリング